# Notebook for data acquisition functions


#### The first fctn take he target year and a filepath as argument to download the data from a specific season and put it in a certain path

In [10]:
import pandas as pd
import requests

#def get_season_data(year, filepath):
    

In [18]:
GAME_ID = 2017020608
PATH = '../ift6758/data'
r = requests.get(f"https://statsapi.web.nhl.com/api/v1/game/{GAME_ID}/feed/live/")
#r.json()
with open("../ift6758/data/test.json", 'wb') as f:
    f.write(r.content)